<a href="https://colab.research.google.com/github/charlie-aashutosh/Deep-Learning/blob/master/Sentiment_Classification_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
from keras.datasets import imdb

In [5]:
#setting vocabulary size and loading the test and train data
vocabulary_size = 5000

(X_train,y_train),(X_test,y_test)=imdb.load_data(num_words=vocabulary_size)
print('loaded dataset with {} training samples and {} test samples'.format(len(X_train),len(X_test)))


loaded dataset with 25000 training samples and 25000 test samples


In [14]:
#getting a closer loook to our dataset
print('---review---')
print(X_train[10])
print('---label---')
print(y_train[10])

---review---
[1, 785, 189, 438, 47, 110, 142, 7, 6, 2, 120, 4, 236, 378, 7, 153, 19, 87, 108, 141, 17, 1004, 5, 2, 883, 2, 23, 8, 4, 136, 2, 2, 4, 2, 43, 1076, 21, 1407, 419, 5, 2, 120, 91, 682, 189, 2818, 5, 9, 1348, 31, 7, 4, 118, 785, 189, 108, 126, 93, 2, 16, 540, 324, 23, 6, 364, 352, 21, 14, 9, 93, 56, 18, 11, 230, 53, 771, 74, 31, 34, 4, 2834, 7, 4, 22, 5, 14, 11, 471, 9, 2, 34, 4, 321, 487, 5, 116, 15, 2, 4, 22, 9, 6, 2286, 4, 114, 2679, 23, 107, 293, 1008, 1172, 5, 328, 1236, 4, 1375, 109, 9, 6, 132, 773, 2, 1412, 8, 1172, 18, 2, 29, 9, 276, 11, 6, 2768, 19, 289, 409, 4, 2, 2140, 2, 648, 1430, 2, 2, 5, 27, 3000, 1432, 2, 103, 6, 346, 137, 11, 4, 2768, 295, 36, 2, 725, 6, 3208, 273, 11, 4, 1513, 15, 1367, 35, 154, 2, 103, 2, 173, 7, 12, 36, 515, 3547, 94, 2547, 1722, 5, 3547, 36, 203, 30, 502, 8, 361, 12, 8, 989, 143, 4, 1172, 3404, 10, 10, 328, 1236, 9, 6, 55, 221, 2989, 5, 146, 165, 179, 770, 15, 50, 713, 53, 108, 448, 23, 12, 17, 225, 38, 76, 4397, 18, 183, 8, 81, 19, 12, 45

#here we can see that revies is stored as a sequence of integers ~ word ID is pre assigned to individual words and label is 0 for negative and 1 for positive statment or review

In [16]:
word2id=imdb.get_word_index()
id2word={i:word for word, i in word2id.items()}
print('---review with words---')
print([id2word.get(i,'') for i in X_train[10]])
print('---label---')
print(y_train[10])

---review with words---
['the', 'clear', 'fact', 'entertaining', 'there', 'life', 'back', 'br', 'is', 'and', 'show', 'of', 'performance', 'stars', 'br', 'actors', 'film', 'him', 'many', 'should', 'movie', 'reasons', 'to', 'and', 'reading', 'and', 'are', 'in', 'of', 'scenes', 'and', 'and', 'of', 'and', 'out', 'compared', 'not', 'boss', 'yes', 'to', 'and', 'show', 'its', 'disappointed', 'fact', 'raw', 'to', 'it', 'justice', 'by', 'br', 'of', 'where', 'clear', 'fact', 'many', 'your', 'way', 'and', 'with', 'city', 'nice', 'are', 'is', 'along', 'wrong', 'not', 'as', 'it', 'way', 'she', 'but', 'this', 'anything', 'up', "haven't", 'been', 'by', 'who', 'of', 'choices', 'br', 'of', 'you', 'to', 'as', 'this', "i'd", 'it', 'and', 'who', 'of', 'shot', "you'll", 'to', 'love', 'for', 'and', 'of', 'you', 'it', 'is', 'sequels', 'of', 'little', 'quest', 'are', 'seen', 'watched', 'front', 'chemistry', 'to', 'simply', 'alive', 'of', 'chris', 'being', 'it', 'is', 'say', 'easy', 'and', 'cry', 'in', 'chemis

In [20]:
#maximum and minimum review length
print('maximum review length:{}'.format(len(max((X_train+X_test), key=len))))

maximum review length:2697


In [22]:
print('minimum review length:{}'.format(len(min((X_train+X_test), key=len))))

minimum review length:70


#padding the sequence ~ all input documents must have the same length while we feed them into the RNN, so we can limit the max review lenght to max_words by truncating longer reviews and padding shorter reviews with null values

In [23]:
from keras.preprocessing import sequence

In [24]:
max_words=500
X_train=sequence.pad_sequences(X_train, maxlen=max_words)
X_test=sequence.pad_sequences(X_test,maxlen=max_words)

In [25]:
#designing our RNN model

Our input layer is sequence of words(integer word ID) of maximum length = max_words, output is a binary classifier label 0 or 1

In [27]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
#training and evaluation of our model

We need to compile our model by specifying the loss function and optimizer we want to use while training as well as some evaluation metrices which we would like to measure

In [31]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
#while training we need to specify the batch_size and the number of training epochs

In [33]:
batch_size=64
num_epochs=3

X_valid,y_valid=X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2,y_train2, validation_data=(X_valid,y_valid),batch_size=batch_size,epochs=num_epochs)

Epoch 1/3
390/390 [==============================] - 230s 591ms/step - loss: 0.4896 - accuracy: 0.7534 - val_loss: 0.2884 - val_accuracy: 0.8594
Epoch 2/3
390/390 [==============================] - 233s 598ms/step - loss: 0.2949 - accuracy: 0.8819 - val_loss: 0.3354 - val_accuracy: 0.8438
Epoch 3/3
390/390 [==============================] - 238s 609ms/step - loss: 0.2465 - accuracy: 0.9034 - val_loss: 0.2127 - val_accuracy: 0.9219


In [34]:
#let us see how well the model performs in case of test data
scores=model.evaluate(X_test, y_test)
print('Test_accuracy:',scores[1])

782/782 [==============================] - 73s 93ms/step - loss: 0.3077 - accuracy: 0.8758
Test_accuracy: 0.8758000135421753
